### Hybrid Search

In [4]:
%pip install --upgrade --quiet  pinecone pinecone-text pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [5]:
PINECONE_API_KEY="pcsk_SJjUx_2GVmteUhkfkNmuof16BvR6hG9cSpo3DZdDPV2CKHLGqpqZ4Wmad35KYQay1whzX"

In [6]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [7]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name = "hybrid-search-langchain-pinecone"
##Intitialize pinecone client
pc=Pinecone(api_key=PINECONE_API_KEY)

#create index
if index_name not in pc.list_indexes().names(): ##check if index already exists
    pc.create_index(name=index_name,dimension=384,metric="dotproduct",spec=ServerlessSpec(cloud='aws',region="us-east-1")) ##cosine similarity

c:\hybrid_search\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
index=pc.Index(index_name)
index

In [11]:
##vector embeddings and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default() ##initialize BM25 encoder tf idf
bm25_encoder

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\radhi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [14]:
sentences=["I am a student","I am a teacher","I am a doctor","I am a lawyer","I am a nurse","I am a engineer","I am a scientist","I am a researcher","I am a developer","I am a designer"]
bm25_encoder.fit(sentences) ##fit the encoder with the sentences or applying tfidf

##Store the values to json file
bm25_encoder.dump("bm25_values.json")

##Load the values from json file
bm25_encoder.load("bm25_values.json")

100%|██████████| 10/10 [00:00<00:00, 141.56it/s]


In [16]:
retriever=PineconeHybridSearchRetriever(index=index,embeddings=embeddings,sparse_encoder=bm25_encoder) ##Supoprts voth vector embedding and sparse matrix
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000022209B84DC0>, index=<pinecone.data.index.Index object at 0x00000222089D4AC0>)

In [17]:
retriever.add_texts(sentences) ##add the sentences to the retriever

100%|██████████| 1/1 [00:01<00:00,  1.68s/it]


In [18]:
retriever.invoke("What is your profession?") ##invoke the retriever with the query

[Document(metadata={'score': 0.260198772}, page_content='I am a doctor'),
 Document(metadata={'score': 0.249755234}, page_content='I am a nurse'),
 Document(metadata={'score': 0.245562643}, page_content='I am a teacher'),
 Document(metadata={'score': 0.242390066}, page_content='I am a lawyer')]